In [1]:
from google.colab import drive
drive.mount('/content/drive')

root = '/content/drive/My Drive/surely_not_sarcasm_data/'

Mounted at /content/drive


In [ ]:
!pip install contractions
!cp -r '$root/utils' .

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import keras
import pickle
import nltk
import time

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from nltk.sentiment import SentimentIntensityAnalyzer

from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Dense, Embedding, GRU, LSTM, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Attention, MultiHeadAttention
from tensorflow.keras.layers import Concatenate, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import L1L2

from utils.preprocessing import WordEmbeddings
from utils.evaluation import plot_history, evaluate, show_errors

from numpy.linalg import norm

sns.set_style('whitegrid');
sns.set_context('notebook', font_scale=1.1);

In [5]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [6]:
data = pickle.load(open(root+'all_preprocessed_data_2105', 'rb'))
df = pd.DataFrame(data).reset_index(drop=True)

In [7]:
X = df[['raw_comment', 'raw_parent', 'comment', 'parent']]
y = df['label'].values

# helper functions + get auxiliary features

In [9]:
def get_polarity(column):
    sia = SentimentIntensityAnalyzer()
    polarity = column.apply(lambda x: [sia.polarity_scores(w)['compound'] for w in x.split()])
    min_pol, max_pol = polarity.apply(min), polarity.apply(max)
    abs_diff, mean_pol = polarity.apply(lambda x: abs(min(x)-max(x))), polarity.apply(np.mean)
    return min_pol, max_pol, abs_diff, mean_pol

In [10]:
def get_auxiliary_features_df(column):
    aux_df = pd.DataFrame()

    all_caps = column.str.count('xxallcapsxx')
    ntokens = (
        column
        .str.replace('xxallcapsxx', '', regex=False)
        .str.replace('xxelongxx', '', regex=False)
        .str.replace('xxnumberxx', '', regex=False)
        .str.replace('xxurlxx', '', regex=False)
        .str.replace('xxuserxx', '', regex=False)
        .str.replace('[^a-zA-Z ]', '', regex=True)
        .str.split().str.len()
    )
    aux_df['all_caps_ratio'] = np.where(ntokens != 0, all_caps/ntokens, 0)

    aux_df['n_exl_marks'] = column.str.count(r'\!')
    aux_df['n_q_marks'] = column.str.count(r'\?')
    aux_df['n_ellipsis'] = column.str.count(r'\.\.\.')
    aux_df['n_quot'] = column.str.count(r'\"')
    aux_df['n_stars'] = column.str.count(r'\*')

    aux_df['n_pos_emo'] = column.str.count(r'\:\)|\;\)|\:D|<3')
    aux_df['n_neg_emo'] = column.str.count(r'\:\(|\:\/|\;\(')

    aux_df['min_pol'], aux_df['max_pol'], aux_df['abs_diff_pol'], aux_df['mean_pol'] = get_polarity(column)
    return aux_df

In [11]:
# aux_df = get_auxiliary_features_df(df['comment'])

# with open(root+'data/aux_df', 'wb') as file:
#     pickle.dump(aux_df, file)

In [12]:
def get_emotion_embeddings(emb_matrix, vocab, seed_words):
    length = emb_matrix.shape[1]
    seed_word_vec = WordEmbeddings().get_glove_vectors(tokens=seed_words, vec_len=length, root=root)
    emotion_emb = []

    for i, word in enumerate(vocab):
        word_vec = emb_matrix[i, :]
        if word_vec.any():
            emotion_vec = []
            for seed_word in seed_words:
                emotion_vec.append(cosine_similarity(word_vec, seed_word_vec[seed_word]))
            emotion_emb.append(emotion_vec)
        else:
            emotion_emb.append([0]*len(seed_words))

    return np.array(emotion_emb)

In [13]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (norm(vec1)*norm(vec2))

# define models

In [14]:
def rnn_gru(tokenizer_layer, embedding_layer):
    model = Sequential([
        Input(shape=(1,), dtype=tf.string),
        tokenizer_layer,
        embedding_layer,
        GRU(100, dropout=0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def bi_gru(tokenizer_layer, embedding_layer):
    model = Sequential([
        Input(shape=(1,), dtype=tf.string),
        tokenizer_layer,
        embedding_layer,
        Bidirectional(GRU(100, dropout=0.3)),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def rnn_lstm(tokenizer_layer, embedding_layer):
    model = Sequential([
        Input(shape=(1,), dtype=tf.string),
        tokenizer_layer,
        embedding_layer,
        LSTM(100, dropout=0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def bi_gru_aux(tokenizer_layer, embedding_layer):
    aux_shape = pickle.load(open(root+'data/aux_df', 'rb')).shape[1]

    comment_input =  Input(shape=(1,), name='comment', dtype=tf.string)
    aux_input = Input(shape=(aux_shape,), name='aux')

    tokenize_comment = tokenizer_layer(comment_input)
    emb = embedding_layer(tokenize_comment)
    gru = Bidirectional(GRU(100, dropout=0.3))(emb)

    x = Concatenate()([gru, aux_input])
    x = BatchNormalization()(x)
    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, aux_input], outputs=[output])
    return model

In [ ]:
def bi_gru_emo(tokenizer_layer,  embedding_layer):
    comment_input = Input(shape=(1,), dtype=tf.string)
    tokenize_comment = tokenizer_layer(comment_input)

    word_rep = embedding_layer[0](tokenize_comment)
    word_rep = Bidirectional(GRU(100, dropout=0.3))(word_rep)

    emo_rep = embedding_layer[1](tokenize_comment)
    emo_rep = GRU(32, dropout=0.3)(emo_rep)

    x = Concatenate()([word_rep, emo_rep])
    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input], outputs=[output])
    return model

In [ ]:
def gru_context(tokenizer_layer, embedding_layer):
    comment_input =  Input(shape=(1,), name='comment', dtype=tf.string)
    parent_input = Input(shape=(1,), name='parent', dtype=tf.string)

    tokenize_comment = tokenizer_layer[0](comment_input)
    tokenize_parent = tokenizer_layer[1](parent_input)

    emb_comment = embedding_layer[0](tokenize_comment)
    emb_parent = embedding_layer[1](tokenize_parent)

    gru_comment = GRU(100, dropout=0.3)(emb_comment)
    gru_parent = GRU(100, dropout=0.3)(emb_parent)

    x = Concatenate()([gru_comment, gru_parent])
    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, parent_input], outputs=[output])
    return model

In [ ]:
def bi_gru_context(tokenizer_layer, embedding_layer):
    comment_input =  Input(shape=(1,), name='comment', dtype=tf.string)
    parent_input = Input(shape=(1,), name='parent', dtype=tf.string)

    tokenize_comment = tokenizer_layer[0](comment_input)
    tokenize_parent = tokenizer_layer[1](parent_input)

    emb_comment = embedding_layer[0](tokenize_comment)
    emb_parent = embedding_layer[1](tokenize_parent)

    gru_comment = Bidirectional(GRU(100, dropout=0.3))(emb_comment)
    gru_parent = Bidirectional(GRU(100, dropout=0.3))(emb_parent)

    x = Concatenate()([gru_comment, gru_parent])
    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, parent_input], outputs=[output])
    return model

In [ ]:
def cnn(tokenizer_layer, embedding_layer):
    model = Sequential([
        Input(shape=(1,), dtype=tf.string),
        tokenizer_layer,
        embedding_layer,

        Conv1D(64, 2, activation='relu', kernel_initializer='he_normal', padding='same'),
        BatchNormalization(),
        Conv1D(64, 2, activation='relu', kernel_initializer='he_normal', padding='same'),
        BatchNormalization(),
        MaxPooling1D(2, padding='same'),
        Dropout(0.3),

        Conv1D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same'),
        BatchNormalization(),
        Conv1D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same'),
        BatchNormalization(),
        
        GlobalMaxPooling1D(),
        Dense(100, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    return model

In [ ]:
def cnn_lstm_dnn(tokenizer_layer, embedding_layer):
    model = Sequential([
        Input(shape=(1,), dtype=tf.string),
        tokenizer_layer,
        embedding_layer,
        Conv1D(64, 2, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(),
        Conv1D(128, 3, activation='relu', kernel_initializer='he_normal'),
        BatchNormalization(),
        LSTM(100, dropout=0.25, return_sequences=True),
        LSTM(100, dropout=0.25),
        Dense(50, activation='relu', kernel_initializer='he_normal'),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def mha_self_attention(tokenizer_layer, embedding_layer):
    input_layer = keras.Input(shape=(1,), dtype=tf.string)
    tokenize_comment = tokenizer_layer(input_layer)

    embeddings = embedding_layer(tokenize_comment)
    mask = embedding_layer.compute_mask(tokenize_comment)[:, tf.newaxis]

    gru = Bidirectional(GRU(100, dropout=0.3, return_sequences=True))(embeddings)
    mha, _ = MultiHeadAttention(3, 64, name='attention')(gru, gru, attention_mask=mask,
                                                        return_attention_scores=True)
    x = GlobalMaxPooling1D()(mha)

    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[input_layer], outputs=[output])
    return model

In [ ]:
def build_cnn(tok_comment, emb_layer):
    x = emb_layer(tok_comment)

    x = Conv1D(64, 2, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv1D(64, 2, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2, padding='same')(x)
    x = Dropout(0.3)(x)

    x = Conv1D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv1D(128, 3, activation='relu', kernel_initializer='he_normal', padding='same')(x)
    x = BatchNormalization()(x)

    x = GlobalMaxPooling1D()(x)

    return x

In [ ]:
def cnn_context(tokenizer_layer, embedding_layer):
    comment_input = Input(shape=(1,), name='comment', dtype=tf.string)
    parent_input = Input(shape=(1,), name='parent', dtype=tf.string)

    tokenize_comment = tokenizer_layer[0](comment_input)
    tokenize_parent = tokenizer_layer[1](parent_input)

    comment_cnn = build_cnn(tokenize_comment, embedding_layer[0])
    parent_cnn = build_cnn(tokenize_parent, embedding_layer[1])

    x = Concatenate()([comment_cnn, parent_cnn])
    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, parent_input], outputs=[output])
    return model

In [ ]:
def gru_cnn_context(tokenizer_layer, embedding_layer):
    comment_input =  keras.Input(shape=(1,), name='comment', dtype=tf.string)
    parent_input = keras.Input(shape=(1,), name='parent', dtype=tf.string)

    tokenize_comment = tokenizer_layer[0](comment_input)
    tokenize_parent = tokenizer_layer[1](parent_input)

    emb_comment = embedding_layer[0](tokenize_comment)
    gru_comment = GRU(100, dropout=0.3)(emb_comment)

    parent_cnn = build_cnn(tokenize_parent, embedding_layer[1])

    x = Concatenate()([gru_comment, parent_cnn])
    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, parent_input], outputs=[output])
    return model

In [ ]:
def mha_parent_comment(tokenizer_layer, embedding_layer):
    comment_input =  Input(shape=(1,), name='comment', dtype=tf.string)
    parent_input = Input(shape=(1,), name='parent', dtype=tf.string)

    tokenize_comment = tokenizer_layer[0](comment_input)
    tokenize_parent = tokenizer_layer[1](parent_input)

    emb_comment = embedding_layer[0](tokenize_comment)
    emb_parent = embedding_layer[1](tokenize_parent)

    maxlen, maxlen_par = embedding_layer[0].input_length, embedding_layer[1].input_length

    mask1 = tf.squeeze(embedding_layer[0].compute_mask(tokenize_comment))
    mask2 = tf.squeeze(embedding_layer[1].compute_mask(tokenize_parent))
    mask = tf.reshape(mask2, [-1, maxlen_par, 1]) & tf.reshape(mask1, [-1, 1, maxlen])

    gru_comment = GRU(100, dropout=0.3, return_sequences=True)(emb_comment)
    gru_parent = GRU(100, dropout=0.3, return_sequences=True)(emb_parent)
    mha, _ = MultiHeadAttention(3, 64, name='attention')(gru_parent, gru_comment,
                                                        attention_mask=mask, return_attention_scores=True)
    x = GlobalMaxPooling1D()(mha)

    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, parent_input], outputs=[output])
    return model

In [ ]:
def mha_emo(tokenizer_layer, embedding_layer):
    comment_input =  Input(shape=(1,), name='comment', dtype=tf.string)
    parent_input = Input(shape=(1,), name='parent', dtype=tf.string)

    tokenize_comment = tokenizer_layer[0](comment_input)
    tokenize_parent = tokenizer_layer[1](parent_input)

    emb_comment = embedding_layer[0](tokenize_comment)
    emb_parent = embedding_layer[1](tokenize_parent)

    emotion_comment = embedding_layer[2](tokenize_comment)
    emotion_parent = embedding_layer[3](tokenize_parent)

    maxlen, maxlen_par = embedding_layer[2].input_length, embedding_layer[3].input_length

    mask1 = tf.squeeze(embedding_layer[2].compute_mask(tokenize_comment))
    mask2 = tf.squeeze(embedding_layer[3].compute_mask(tokenize_parent))
    mask = tf.reshape(mask2, [-1, maxlen_par, 1]) & tf.reshape(mask1, [-1, 1, maxlen])

    gru_emo_comment = GRU(32, dropout=0.3, return_sequences=True)(emotion_comment)
    gru_emo_parent = GRU(32, dropout=0.3, return_sequences=True)(emotion_parent)
    mha, _ = MultiHeadAttention(3, 64, name='attention')(gru_emo_parent, gru_emo_comment,
                                                         attention_mask=mask, return_attention_scores=True)
    mha = GlobalMaxPooling1D()(mha)

    gru_comment = GRU(100, dropout=0.3)(emb_comment)
    gru_parent = GRU(100, dropout=0.3)(emb_parent)

    x = Concatenate()([gru_comment, gru_parent, mha])

    x = Dense(100, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=[comment_input, parent_input], outputs=[output])
    return model

# pseudo-validation

In [33]:
random_seeds = [83, 91, 61]

In [34]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [35]:
def pseudo_validation(define_model, X, y, random_seeds, name, context=False, aux=False, emo=False, all=False):
    highest_acc = 0
    results = []

    for seed in random_seeds:
        X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.3, random_state=seed)
        X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, stratify=y_val, test_size=0.5, random_state=seed)

        vocab_size = 10000
        input_dim = vocab_size + 2 # one for padding, one for oov token
        
        maxlen = int(X_train['comment'].str.split().str.len().quantile(0.9))
        tokenizer_layer = TextVectorization(max_tokens=input_dim, output_sequence_length=maxlen, standardize=None)
        tokenizer_layer.adapt(X_train['comment'])

        vocab_com = tokenizer_layer.get_vocabulary()

        vec_len = 100
        wordemb = WordEmbeddings()
        emb_matrix_comment = wordemb.get_glove_embeddings(input_dim, vec_len, vocab_com, root=root)

        train_data = X_train['comment'].values
        val_data = X_val['comment'].values
        test_data = X_test['comment'].values

        embedding_layer = Embedding(
            input_dim=input_dim, output_dim=vec_len, input_length=maxlen,
            mask_zero=True, weights=[emb_matrix_comment], trainable=True,
            embeddings_regularizer=L1L2(l1=3e-6, l2=3e-6)
        )

        tok_layer = tokenizer_layer
        emb_layer = embedding_layer

        if context:
            maxlen_par = int(X_train['parent'].str.split().str.len().quantile(0.8))
            tokenizer_par_layer = TextVectorization(max_tokens=input_dim, output_sequence_length=maxlen_par, standardize=None)
            tokenizer_par_layer.adapt(X_train['parent'])

            vocab_par = tokenizer_par_layer.get_vocabulary()
            emb_matrix_parent = wordemb.get_glove_embeddings(input_dim, vec_len, vocab_par, root=root)

            train_data = {'comment': X_train['comment'].values, 'parent': X_train['parent'].values}
            val_data = {'comment': X_val['comment'].values, 'parent': X_val['parent'].values}
            test_data = {'comment': X_test['comment'].values, 'parent': X_test['parent'].values}

            embedding_par_layer = Embedding(
                input_dim=input_dim, output_dim=vec_len, input_length=maxlen_par,
                mask_zero=True, weights=[emb_matrix_parent], trainable=True,
                embeddings_regularizer=L1L2(l1=3e-6, l2=3e-6)
            )

            tok_layer = [tokenizer_layer, tokenizer_par_layer]
            emb_layer = [embedding_layer, embedding_par_layer]

            if emo:
                seed_words = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'good', 'bad']
                emotion_emb_com = get_emotion_embeddings(emb_matrix_comment, vocab_com, seed_words)
                emotion_emb_par = get_emotion_embeddings(emb_matrix_parent, vocab_par, seed_words)

                emb_emo_layer_com = Embedding(
                    input_dim=input_dim, output_dim=len(seed_words), input_length=maxlen,
                    mask_zero=True, weights=[emotion_emb_com], trainable=False
                )

                emb_emo_layer_par = Embedding(
                    input_dim=input_dim, output_dim=len(seed_words), input_length=maxlen_par,
                    mask_zero=True, weights=[emotion_emb_par], trainable=False
                )

                emb_layer += [emb_emo_layer_com, emb_emo_layer_par]

        if aux:
            aux_df = pickle.load(open(root+'data/aux_df', 'rb'))
            aux_train, aux_val, _, aux_y = train_test_split(aux_df, y, stratify=y, test_size=0.3, random_state=seed)
            aux_val, aux_test, _, _ = train_test_split(aux_val, aux_y, stratify=aux_y, test_size=0.5, random_state=seed)

            scaler = StandardScaler()
            aux_train = scaler.fit_transform(aux_train)
            aux_val = scaler.transform(aux_val)
            aux_test = scaler.transform(aux_test)

            train_data = {'comment': X_train['comment'].values, 'aux': aux_train}
            val_data = {'comment': X_val['comment'].values, 'aux': aux_val}
            test_data = {'comment': X_test['comment'].values, 'aux': aux_test}

        if all:
            train_data = {'comment': X_train['comment'].values, 'parent': X_train['parent'].values, 'aux': aux_train}
            val_data = {'comment': X_val['comment'].values, 'parent': X_val['parent'].values, 'aux': aux_val}
            test_data = {'comment': X_test['comment'].values, 'parent': X_test['parent'].values, 'aux': aux_test}

        if not context and emo:
            seed_words = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'good', 'bad']
            emotion_emb = get_emotion_embeddings(emb_matrix_comment, vocab_com, seed_words)

            emb_layer_emo = Embedding(
                input_dim=input_dim, output_dim=len(seed_words), input_length=maxlen,
                mask_zero=True, weights=[emotion_emb], trainable=False
            )

            emb_layer = [embedding_layer, emb_layer_emo]

        model = define_model(tok_layer, emb_layer)

        binary_crossentropy = tf.keras.metrics.BinaryCrossentropy(name='bc')
        es = EarlyStopping(monitor='val_bc', min_delta=1e-5, patience=5, restore_best_weights=True)
        timeit = TimeHistory()

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[binary_crossentropy, 'accuracy'])
        start_time = time.time()
        fit = model.fit(train_data, y_train, batch_size=128, epochs=100, callbacks=[es, timeit],
                        validation_data=(val_data, y_val), verbose=0)
        end_time = time.time()

        y_pred = model.predict(test_data).round().flatten()
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        times = timeit.times
        n_epochs = len(times)
        avg_time_per_epoch = np.mean(times)
        total_time = np.sum(times)
        total_time_2 = end_time - start_time

        results.append([seed, acc, prec, rec, f1, n_epochs, avg_time_per_epoch, total_time, total_time_2])
        columns = ['seed', 'acc', 'prec', 'rec', 'f1', 'n_epochs', 'avg_time_per_epoch', 'total_time', 'total_time_2']

        res_df = pd.DataFrame(results, columns=columns)
        res_df.to_csv(root+fr'final_results/{name}.csv', index=False)

        if acc > highest_acc:
            highest_acc = acc
            model.save(root+'models/'+name)

        print(fr'run number {len(results)} done')

In [ ]:
pseudo_validation(rnn_gru, X, y, random_seeds, name='gru')

In [ ]:
pd.read_csv(root+'final_results/gru.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.748779,0.762501,0.716941,0.739020,19,104.366604,1982.965479,1983.214535
1,91,0.748991,0.771585,0.701827,0.735055,16,103.975855,1663.613679,1663.869448
2,61,0.749641,0.776121,0.696195,0.733989,19,102.789057,1952.992085,1953.258922


In [ ]:
pseudo_validation(rnn_lstm, X, y, random_seeds, name='lstm')

In [ ]:
pd.read_csv(root+'final_results/lstm.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.749039,0.767894,0.708231,0.736857,16,103.666862,1658.669790,1658.940230
1,91,0.750317,0.762833,0.720846,0.741246,16,100.689848,1611.037573,1611.303041
2,61,0.751349,0.772823,0.706493,0.738171,18,100.723063,1813.015141,1813.290121


In [ ]:
pseudo_validation(bi_gru, X, y, random_seeds, name='bi_gru')

In [ ]:
pd.read_csv(root+'final_results/bi_gru.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.749572,0.776053,0.696109,0.733911,13,143.157055,1861.041711,1861.303700
1,91,0.749949,0.775112,0.698716,0.734934,14,143.638458,2010.938416,2011.196270
2,61,0.751051,0.764638,0.719763,0.741522,13,142.049373,1846.641853,1846.898211


In [ ]:
pseudo_validation(bi_gru_aux, X, y, random_seeds, name='bi_gru_aux', aux=True)

In [ ]:
pd.read_csv(root+'final_results/bi_gru_aux.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.750232,0.757487,0.730404,0.743699,19,162.929157,3095.653976,3095.984940
1,91,0.750242,0.761271,0.723453,0.741880,17,154.792756,2631.476853,2631.814935
2,61,0.752504,0.781275,0.695991,0.736171,17,169.294751,2878.010768,2878.347237


In [ ]:
pseudo_validation(bi_gru_emo, X, y, random_seeds, name='bi_gru_emo', emo=True)

In [ ]:
pd.read_csv(root+'final_results/bi_gru_emo.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.750604,0.766694,0.714839,0.739859,18,206.126299,3710.273390,3710.589808
1,91,0.750833,0.764613,0.719173,0.741197,15,211.755768,3176.336521,3176.649696
2,61,0.751993,0.762449,0.726446,0.744012,16,212.343647,3397.498352,3397.808669


In [ ]:
pseudo_validation(cnn, X, y, random_seeds, name='cnn')

In [ ]:
pd.read_csv(root+'final_results/cnn.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.745160,0.759362,0.711953,0.734894,21,102.903376,2160.970889,2161.259394
1,91,0.744521,0.754787,0.718465,0.736178,23,99.326165,2284.501795,2284.831013
2,61,0.746698,0.769938,0.698008,0.732211,16,97.046949,1552.751185,1553.017965


In [ ]:
pseudo_validation(cnn_lstm_dnn, X, y, random_seeds, name='cnn_lstm_dnn')

In [ ]:
pd.read_csv(root+'final_results/cnn_lstm_dnn.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.748305,0.767085,0.707502,0.736090,23,108.689865,2499.866903,2500.144134
1,91,0.747741,0.765530,0.708564,0.735946,22,108.955865,2397.029035,2397.300882
2,61,0.748906,0.765033,0.712822,0.738005,21,106.962825,2246.219321,2246.499016


In [ ]:
pseudo_validation(mha_self_attention, X, y, random_seeds, name='mha')

In [ ]:
pd.read_csv(root+'final_results/mha.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.750812,0.777059,0.697987,0.735403,20,147.491937,2949.838735,2950.125521
1,91,0.750019,0.769403,0.708467,0.737679,16,147.203290,2355.252648,2355.526321
2,61,0.751024,0.776531,0.699445,0.735975,20,148.721692,2974.433847,2974.712542


In [ ]:
pseudo_validation(gru_context, X, y, random_seeds, name='gru_context', context=True)

In [ ]:
pd.read_csv(root+'final_results/gru_context.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.754186,0.775598,0.709937,0.741316,23,207.922808,4782.224589,4782.706856
1,91,0.754212,0.778094,0.705903,0.740243,17,207.001812,3519.030798,3519.517252
2,61,0.755655,0.767352,0.728302,0.747317,18,205.312967,3695.633411,3696.105552


In [ ]:
pseudo_validation(bi_gru_context, X, y, random_seeds, name='bi_gru_context', context=True)

In [ ]:
pd.read_csv(root+'final_results/bi_gru_context.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.754574,0.782200,0.700314,0.738996,16,312.691006,5003.056101,5003.619976
1,91,0.755617,0.773948,0.716770,0.744262,14,314.068944,4396.965216,4397.534458
2,61,0.756788,0.777835,0.713595,0.744331,19,313.401378,5954.626174,5955.240875


In [ ]:
pseudo_validation(cnn_context, X, y, random_seeds, name='cnn_context', context=True)

In [ ]:
pd.read_csv(root+'final_results/cnn_context.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.750567,0.768361,0.711835,0.739019,20,184.004162,3680.083230,3680.572056
1,91,0.750322,0.767266,0.713026,0.739152,19,179.340979,3407.478599,3407.979060
2,61,0.752100,0.764324,0.723378,0.743287,23,180.114067,4142.623533,4143.141259


In [ ]:
pseudo_validation(gru_cnn_context, X, y, random_seeds, name='gru_cnn_context', context=True)

In [ ]:
pd.read_csv(root+'final_results/gru_cnn_context.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.752217,0.768626,0.716137,0.741454,18,202.408126,3643.346260,3643.915504
1,91,0.752036,0.777881,0.700110,0.736950,18,201.344719,3624.204940,3624.764205
2,61,0.754468,0.769550,0.721018,0.744494,15,200.954834,3014.322508,3014.869469


In [ ]:
pseudo_validation(mha_parent_comment, X, y, random_seeds, name='mha_context', context=True)

In [ ]:
pd.read_csv(root+'final_results/mha_context.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.751913,0.778032,0.699520,0.736690,22,220.252607,4845.557344,4846.111284
1,91,0.751567,0.763375,0.723528,0.742918,24,216.161930,5187.886326,5188.456656
2,61,0.754681,0.772310,0.716877,0.743562,16,222.090845,3553.453516,3554.018699


In [ ]:
pseudo_validation(mha_emo, X, y, random_seeds, name='mha_emo_context', context=True, emo=True)

In [ ]:
pd.read_csv(root+'final_results/mha_emo_context.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.753270,0.778374,0.702792,0.738655,16,267.258484,4276.135744,4276.604334
1,91,0.754101,0.776327,0.708488,0.740858,17,265.698894,4516.881201,4517.351428
2,61,0.755511,0.781079,0.704723,0.740939,18,269.741153,4855.340763,4855.829577


In [4]:
pd.read_csv(root+'final_results/bag_of_words.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.689262,0.726760,0.598803,0.656606,14,61.206024,856.884336,962.223888
1,91,0.688469,0.709676,0.629665,0.667281,22,60.818830,1338.014253,1570.513834
2,61,0.690726,0.720111,0.616074,0.664042,21,60.031352,1260.658399,1414.546453


In [5]:
pd.read_csv(root+'final_results/context_bag_of_words.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.691396,0.724527,0.609841,0.662256,20,39.884813,797.696252,800.002938
1,91,0.693664,0.714730,0.636648,0.673433,13,39.756593,516.835706,517.781237
2,61,0.694286,0.717723,0.632594,0.672475,10,39.870687,398.706867,399.723175


In [39]:
pd.read_csv(root+'final_results/distilbert.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.769577,0.775558,0.753661,0.764452,3,1020.437,3738.852,3738.852
1,91,0.767294,0.763176,0.769848,0.766497,3,1025.463,3755.366,3755.366
2,61,0.767299,0.757703,0.780575,0.768969,3,1021.727,3740.414,3755.366


In [41]:
pd.read_csv(root+'final_results/distilbert_context2.csv')

,seed,acc,prec,rec,f1,n_epochs,avg_time_per_epoch,total_time,total_time_2
0,83,0.778641,0.780685,0.770191,0.775402,3,2284.600,NaN,NaN
1,91,0.778156,0.776442,0.776392,0.776417,3,2279.647,8347.226,8347.226
2,61,0.780615,0.788643,0.762028,0.775107,3,2283.227,8358.546,8358.546
